# Imports

In [1]:
import torch
import torchvision
from torch import nn
from torchvision import models, datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary

import matplotlib.pyplot as plt
import numpy as np

from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from timeit import default_timer as timer
import os
from pathlib import Path
import zipfile
import requests
from datetime import datetime
from PIL import Image
import shutil
import random

2024-04-30 08:36:47.142879: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Going Modular Folder

In [2]:
current_directory = os.getcwd()
parent_folder = os.path.dirname(current_directory)
src_folder = parent_folder + "/src"

going_modular_dir = src_folder + "/going_modular"

if os.path.isdir(going_modular_dir):
  print(f"[INFO] Going Modular directory exists already, skipping creation")
else:
  os.makedirs(going_modular_dir)
  print(f"[INFO] Going Modular directory created")

[INFO] Going Modular directory exists already, skipping creation


# data_setup.py

In [8]:
%%writefile "/Users/kittyli/Desktop/AI and ML/Practices/Garbage Clasification App/src/going_modular/data_setup.py" 
import torch
import torchvision
from torch import nn
from torchvision import models, datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary

import matplotlib.pyplot as plt
import numpy as np

from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from timeit import default_timer as timer
import os
from pathlib import Path
import zipfile
import requests
from datetime import datetime
from PIL import Image
import shutil
import random


default_train_transforms = transforms.Compose([transforms.RandomHorizontalFlip(),
                                               transforms.RandomRotation(30),
                                               transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                                               transforms.GaussianBlur(3),
                                               transforms.ToTensor()])


def create_auto_transforms(model_name: str) -> List[transforms.Compose]:
  '''
  '''
  print("[INFO] create_auto_transforms function incomplete")


def create_dataloaders(train_path: str,
                       val_path: str,
                       test_path: str,
                       batch_size: int,
                       transforms: list,
                       num_workers: int=os.cpu_count()) -> Tuple[DataLoader, DataLoader, DataLoader, List[str]]:
  '''
  This function creates dataloaders for training, validation, and testing datasets.

  Args:
    train_path (str): The path to the training dataset
    val_path (str): The path to the validation dataset
    test_path (str): The path to the testing dataset
    batch_size (int): The batch size for the dataloaders
    transforms (list): A list of transforms to apply to the datasets
    num_workers (int): The number of workers to use for creating dataloaders

  Returns:
    train_dataloader (DataLoader): The training dataloader
    val_dataloader (DataLoader): The validation dataloader
    test_dataloader (DataLoader): The testing dataloader
    class_names (List[str]): A list of class names
  '''
  train_data = ImageFolder(train_path, transform=transforms[0])
  val_data = ImageFolder(val_path, transform=transforms[1])
  test_data = ImageFolder(test_path, transform=transforms[1])

  class_names = train_data.classes

  train_dataloader = DataLoader(dataset=train_data,
                                batch_size=batch_size,
                                num_workers=num_workers,
                                shuffle=True,
                                pin_memory=True)
  
  val_dataloader = DataLoader(dataset=val_data,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              shuffle=False,
                              pin_memory=True)
  
  test_dataloader = DataLoader(dataset=test_data,
                                batch_size=batch_size,
                                num_workers=num_workers,
                                shuffle=False,
                                pin_memory=True)

  return train_dataloader, val_dataloader, test_dataloader, class_names

Overwriting /Users/kittyli/Desktop/AI and ML/Practices/Garbage Clasification App/src/going_modular/data_setup.py


# utils.py

In [3]:
%%writefile "/Users/kittyli/Desktop/AI and ML/Practices/Garbage Clasification App/src/going_modular/utils.py" 
import torch
import torchvision
from torch import nn
from torchvision import models, datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary

import matplotlib.pyplot as plt
import numpy as np

from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from timeit import default_timer as timer
import os
from pathlib import Path
import zipfile
import requests
from datetime import datetime
from PIL import Image
import shutil
import random

from data_setup import create_auto_transforms

def set_seeds(seed: int=42) -> None:
  '''
  This function sets manual seeds for reproducibility.

  Args:
    seed (int): The seed value to use

  Returns:
    None
  '''
  torch.manual_seed(seed)
  torch.cuda.manual_seed(42)
  random.seed(42)


def save_model(model: torch.nn.Module, target_dir: str, model_name: str) -> None:
  '''
  This function saves a model to a specified directory.

  Args:
    model (torch.nn.Module): The model to save
    target_dir (str): The directory to save the model
    model_name (str): The name to save the model as

  Returns:
    None
  '''
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  assert model_name.endswith('.pt') or model_name.endswith(".pth"), "[ERROR] Model name must end with .pt or .pth"
  model_save_path = target_dir_path / model_name

  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(), f=model_save_path)


def eval_model(model: torch.nn.Module,
               test_dataloader: DataLoader,
               loss_fn: torch.nn.Module,
               device: torch.device) -> Tuple[float, float]:
  '''
  This function evaluates a model on a test dataset.

  Args:
    model (torch.nn.Module): The model to evaluate
    test_dataloader (DataLoader): The dataloader for the test dataset
    loss_fn (torch.nn.Module): The loss function to use
    device (torch.device): The device to use for evaluation

  Returns:
    test_loss (float): The loss on the test dataset
    test_acc (float): The accuracy on the test dataset
  '''
  test_loss, test_acc = 0, 0

  model.eval()
  with torch.inference_mode():
    for batch, (X, y) in enumerate(test_dataloader):
      X, y = X.to(device), y.to(device)

      test_logits = model(X)
      loss = loss_fn(test_logits, y)

      test_loss += loss.item()

      test_labels = torch.argmax(test_logits, dim=1)
      test_acc += ((test_labels == y).sum().item()/len(test_labels))

  test_loss /= len(test_dataloader)
  test_acc /= len(test_dataloader)

  return test_loss, test_acc


def custom_eval(model: torch.nn.Module,
                model_name: str,
                image_path: str,
                class_names: List[str],
                device: torch.device) -> Tuple[str, float]:
  '''
  This function predicts on a custom image.

  Args:
    model (torch.nn.Module): The model to use for prediction
    model_name (str): The name of the model
    image_path (str): The path to the image
    class_names (list): The class names for the model
    device (torch.device): The device to use for prediction

  Returns:
    pred_class (str): The predicted class
    pred_prob (float): The predicted probability
  '''

  auto_transforms = create_auto_transforms(model_name)

  image = Image.open(image_path)
  transformed_image = auto_transforms(image).unsqueeze(dim=1).to(device)

  model.to(device)
  model.eval()

  with torch.inference_mode():
    pred_logits = model(transformed_image)
    pred_probs = torch.softmax(pred_logits, dim=1)
    pred_label = pred_logits.argmax(dim=1)

    pred_class = class_names[pred_label]
    pred_prob = pred_probs.max()
  
  return pred_class, pred_prob

Overwriting /Users/kittyli/Desktop/AI and ML/Practices/Garbage Clasification App/src/going_modular/utils.py
